# Check Ground Track Velocity Against Azimuth Spacing.

In [1]:
%load_ext autoreload
%autoreload 2
import nisarhdf
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import rioxarray
from osgeo import gdal
import getpass

## Open File as RIFG object

This is the new product from Heresh, June 10, 2025.

In [15]:
RIFGFile = 'product.h5' # Modify to local file path
myRIFG = nisarhdf.nisarRIFGHDF()
# For now includ orbits since they are not in the test products
myRIFG.openHDF(RIFGFile , referenceOrbitXML=None, secondaryOrbitXML=None)
#,
#               referenceOrbit=49672, secondaryOrbit=49879)


ref orbit None
RIFG referenceZeroDopplerStartTime
[698757.51712736]
ref orbit None
RIFG secondaryZeroDopplerStartTime
[698680.64872828]


## Check Azimuth Spacing and Ground Track Velocity Indepdendent of Ground-Track Velocity

Brute force approach by just finding the difference between $(R_{{mid}}, T_{{mid}})$ and $(R_{{mid}}, T_{{mid}} + \delta T_{{mid}})$ in ECEF coordinates, which should give the  azimuth spacing.

In [10]:
myRIFG.orbit = myRIFG.parseStateVectors()
p1 = myRIFG.RTtoECEF(myRIFG.MLCenterRange, myRIFG.MLMidZeroDopplerTime,0 )
p2 = myRIFG.RTtoECEF(myRIFG.MLCenterRange, myRIFG.MLMidZeroDopplerTime + myRIFG.MLZeroDopplerTimeSpacing,0 )
print(f'sceneCenterAlongTrackSpacing = {np.sqrt(np.sum((p1-p2)**2)):.4f}')

sceneCenterAlongTrackSpacing = 22.4201


That used my geocoding routine, so compare it against the data cube, which uses projected coordinates. Scale with appropriate projection scale factor.

In [29]:
x, y = myRIFG.xyCube([myRIFG.MLCenterRange, myRIFG.MLCenterRange], 
                     [myRIFG.MLMidZeroDopplerTime, myRIFG.MLMidZeroDopplerTime + myRIFG.MLZeroDopplerTimeSpacing],
                     [0, 0])
# compute spacing in projected coordinates
azspProj = np.sqrt(np.sum((x[0]-x[1])**2 + (y[0]-y[1])**2))
# Scale to actual meters
import pyproj
lat, lon = myRIFG.CenterLatLon
print(lat, lon)
lengthScale = 1.0 / \
        pyproj.Proj(f"EPSG:{myRIFG.epsg}").get_factors(lon, lat).parallel_scale
azsp = azspProj * lengthScale
print(f'sceneCenterAlongTrackSpacing = {azsp:0.4f}')

34.80890724001087 -117.97139202430002
sceneCenterAlongTrackSpacing = 22.4201


The results are virtually identical, so no issue there. 

## Use Groundtrack Velocity to Compute Azimuth Spacing

Now calculate the spacing as $\delta T_{{mid}} V_{{GT}}$:

In [30]:
azspVGT = myRIFG.groundTrackVelocityCube([myRIFG.MLCenterRange], [myRIFG.MLMidZeroDopplerTime],[0]).item() * myRIFG.MLZeroDopplerTimeSpacing
print(f'Computed from ground track velocity mid-range sceneCenterAlongTrackSpacing = {azspVGT:.4f}')
azspVGT = myRIFG.groundTrackVelocityCube([myRIFG.MLFarRange], [myRIFG.MLMidZeroDopplerTime],[0]).item() * myRIFG.MLZeroDopplerTimeSpacing
print(f'Computed from ground track velocity far range sceneCenterAlongTrackSpacing = {azspVGT:.4f}')
azspVGT = myRIFG.groundTrackVelocityCube([myRIFG.MLNearRange], [myRIFG.MLMidZeroDopplerTime],[0]).item() * myRIFG.MLZeroDopplerTimeSpacing
print(f'Computed from ground track velocity near range sceneCenterAlongTrackSpacing = {azspVGT:.4f}')

Computed from ground track velocity mid-range sceneCenterAlongTrackSpacing = 22.4201
Computed from ground track velocity far range sceneCenterAlongTrackSpacing = 22.4055
Computed from ground track velocity near range sceneCenterAlongTrackSpacing = 22.4356


Now lets check on what the value is in the hdf

In [31]:
print(f"hdf sceneCenterAlongTrackSpacing = {np.array(myRIFG.h5['RIFG']['swaths']['frequencyA']['interferogram']['sceneCenterAlongTrackSpacing']).item():.4f}")

hdf sceneCenterAlongTrackSpacing = 22.5149


Instead of being the tie breaker, we have a completely different number?

In [7]:
!pwd

/home/ian/PycharmProjects/packages/nisarhdf/Notebooks
